<a href="https://colab.research.google.com/github/carranza96/DLSeminar/blob/master/notebooks/IntroDL_TSF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%pip install tensorflow==2.1
%pip install keras-tcn
%matplotlib inline
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPool1D, LSTM, Input, GRU
from tensorflow.keras import backend as K
from tcn import TCN
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.metrics import mean_absolute_error
import datetime
import random
import math

In [0]:
# Fix random seed to allow reproducible experiments
np.random.seed(1)
tf.random.set_seed(1)
random.seed(1)

# Load and inspect USA Temperature data

In [0]:
!mkdir data
!wget -O data/test.csv https://www.dropbox.com/s/8f79c3upf65i1f7/test.csv?dl=1
!wget -O data/train.csv https://www.dropbox.com/s/g2w93sfg0hkmjz0/train.csv?dl=1

In [0]:
# Read training set
train_filename = 'data/train.csv'
with open(train_filename, 'r') as datafile:
  ts_list_train = datafile.readlines()
  ts_list_train = np.asarray([np.asarray(l.rstrip().split(','), dtype=np.float32) for l in ts_list_train])

# Read test set
test_filename = 'data/test.csv'
with open(test_filename, 'r') as datafile:
  ts_list_test = datafile.readlines()
  ts_list_test = np.asarray([np.asarray(l.rstrip().split(','), dtype=np.float32) for l in ts_list_test])

In [0]:
headers = ['San Francisco', 'Los Angeles', 'Las Vegas', 'Dallas', 'Houston', 'Chicago', 'Detroit', 'Miami', 'Toronto', 'Philadelphia', 'New York', 'Boston']
initial_moment = datetime.datetime(2012, 10, 1, 13,0)
colors = plt.get_cmap('Set3', len(headers))

In [0]:
for i in range(len(headers)):
  print("Length of {0} time serie: {1} hours. From {3} to {2}.".format(headers[i], ts_list_train[i].shape[0], initial_moment + datetime.timedelta(hours=ts_list_train[i].shape[0]-1), initial_moment))

print("\nForecasting horizon: {0} hours.".format(ts_list_test[0].shape[0]))

In [0]:
# Lets see how the temperature have evolved over time
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,6))
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%1.2f ºK'))

for i in range(len(headers)):
  x = [initial_moment + datetime.timedelta(hours=j, minutes=0) for j  in range(ts_list_train[i].shape[0])]
  ax.plot(x, ts_list_train[i], label=headers[i], c=colors(i))

ax.legend()
ax.set_title('Temperature')
plt.show()


In [0]:

fig, ax = plt.subplots(nrows=math.ceil(len(headers)/2), ncols=2, figsize=(15,15), sharex=True)

for i in range(len(headers)):
  x = [initial_moment + datetime.timedelta(hours=j, minutes=0) for j  in range(ts_list_train[i].shape[0])]
  ax[i//2][i%2].plot(x, ts_list_train[i], label=headers[i], c=colors(i))
  ax[i//2][i%2].set_title(headers[i])
  ax[i//2][i%2].yaxis.set_major_formatter(ticker.FormatStrFormatter('%1.2f ºK'))
  ax[i//2][i%2].xaxis.set_major_locator(plt.MaxNLocator(5))

plt.show()

# Preprocessing



It is important to scale features before training a neural network. Min-max normalization is a common way of doing this scaling.You could also use any other normalization method that rescales the values into a range of [0,1].

> MIN-MAX NORMALIZATION <br/>
><img src="https://www.oreilly.com/library/view/regression-analysis-with/9781788627306/assets/ffb3ac78-fd6f-4340-aa92-cde8ae0322d6.png" alt="min max formula" width="185"/>
><br/>
><img src="https://www.researchgate.net/publication/282541174/figure/fig1/AS:307388692353061@1450298583749/Min-max-method-of-normalization.png" alt="min max formula" width="350"/>







In [0]:
def normalize(ts, norm_params):
  """
  Apply min-max normalization
  :param data: time series
  :param norm_params: tuple with params mean, std, max, min
  :return: normalized time series
  """
  return (ts - norm_params['min']) / (norm_params['max'] - norm_params['min'])

In [0]:
#Let's normalize the data.
ts_list_train_norm = []
# Save training norm params in order to use it for the test data 
norm_params_list = []

for ts in ts_list_train:
  norm_params = {}
  norm_params['mean'] = ts.mean()
  norm_params['std'] = ts.std()
  norm_params['max'] = ts.max()
  norm_params['min'] = ts.min()
  norm_params_list.append(norm_params)

  ts_norm = normalize(ts, norm_params)
  ts_list_train_norm.append(ts_norm)


In [0]:
# Lets normalize the test data
ts_list_test_norm = []
for i, ts in enumerate(ts_list_test):
  norm_params = norm_params_list[i]
  ts_norm = normalize(ts, norm_params)
  ts_list_test_norm.append(ts_norm)


In [0]:
# This is how the time series look like now. 
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,6))

for i in range(len(headers)):
  x = [initial_moment - datetime.timedelta(hours=i, minutes=0) for i  in range(ts_list_train_norm[i].shape[0])]
  ax.plot(x, ts_list_train_norm[i], label=headers[i], c=colors(i))

ax.legend()
ax.set_title('Temperature (Normalized data)')
plt.show()

Let's now create the data for the model. The model will be given the last 120 hours (5 days) recorded price values, and needs to learn to predict the price at the next 24 hours (1 day). 

In [0]:
past_history = 240
forecast_horizon = 24


<img src="https://cdn-images-1.medium.com/max/1000/1*v5_QpzkQfufVogeCY9eaOw.png" alt="RNN data input" width="500"/>


In [0]:
x_train, y_train = [], []
x_test, y_test = [], []

for i, ts in enumerate(ts_list_train_norm):
  # Train data
  ts_x_train, ts_y_train = [], []
  for j in range(0, ts.shape[0] - forecast_horizon + 1):
      indices = range(j - past_history, j, 1)
      # Reshape data from (past_history,) to (past_history, 1)
      ts_x_train.append(np.reshape(ts[indices], (past_history, 1)))
      ts_y_train.append(ts[j:j + forecast_horizon])
  x_train.extend(np.asarray(ts_x_train))
  y_train.extend(np.asarray(ts_y_train))
  # Test data
  ts_x_test = np.reshape(np.asarray(ts[-past_history:]), (past_history, 1))
  ts_y_test=  ts_list_test_norm[i]
  x_test.extend(np.asarray([ts_x_test]))
  y_test.extend(np.asarray([ts_y_test]))
  

x_train, y_train = np.asarray(x_train), np.asarray(y_train)
x_test, y_test = np.asarray(x_test), np.asarray(y_test, dtype='float32')

print("TRAINING DATA")
print("Input shape", x_train.shape)
print("Output_shape", y_train.shape)
print()
print("TEST DATA")
print("Input shape", x_test.shape)
print("Output_shape", y_test.shape)



Visualize some examples

In [0]:
def show_plot(x, y, y_pred=None):
  fig, ax = plt.subplots(1,1, figsize=(15,5))
  ax.plot(x, 'o-', c=colors(3), markersize=3.5, label='History')
  ax.plot([m for m in range(x.shape[0],x.shape[0]+y.shape[0])], y, 'x-', c=colors(0), markersize=3.5, label='True Future')
  if y_pred is not None:
    ax.plot([m for m in range(x.shape[0],x.shape[0]+y.shape[0])], y_pred, 'o-', c=colors(2), markersize=3.5, label='Model prediction')
  ax.legend()

In [0]:
for _ in range(3):
  random_index = random.randint(0,x_train.shape[0])
  show_plot(x_train[random_index],y_train[random_index])


In [0]:
from collections import defaultdict
results = defaultdict(lambda: {})

# Long-Short Term Memory (LSTM) Neural Network 

In [0]:
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, LSTM, Input

In [0]:
# Create model with one LSTM layer with 64 units
inp = Input(shape=x_train.shape[-2:])
x = LSTM(64)(inp)
x = Dense(forecast_horizon)(x)
model = keras.Model(inputs=inp, outputs=x)

model.compile(optimizer='adam', loss='mae')
model.summary()

Lets train the model

In [0]:
history = model.fit(x_train, y_train,
          batch_size=256,
          epochs=5,
          verbose=1,
          validation_data=(x_test, y_test))

In [0]:
## Visualize training stats 
plt.figure()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Training', 'Validation'])
plt.show()

Let's see the predictions

In [0]:
predictions = model.predict(x_test)
mae = mean_absolute_error(y_test, predictions)
results['MAE']['SimpleLSTM'] = mae
results['Y_PRED']['SimpleLSTM'] = predictions
pd.DataFrame(results)['MAE']

In [0]:
for x, y, y_pred in zip(x_test, y_test, predictions):
  show_plot(x,y,y_pred)

In [0]:
# We can create a more-sofisticated model by stacking LSTM layers. 
inp = Input(shape=x_train.shape[-2:])
x = LSTM(64, return_sequences=True)(inp)
x = LSTM(32, return_sequences=False)(x)
x = Dense(64)(x)
x = Dense(forecast_horizon)(x)
model = keras.Model(inputs=inp, outputs=x)

model.compile(optimizer='adam', loss='mae')
print(model.summary())

history = model.fit(x_train, y_train,
          batch_size=256,
          epochs=5,
          verbose=1,
          validation_data=(x_test, y_test))

plt.figure()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Training', 'Validation'])

predictions = model.predict(x_test)
mae = mean_absolute_error(y_test, predictions)
results['MAE']['LSTM'] = mae
results['Y_PRED']['LSTM'] = predictions
print(pd.DataFrame(results)['MAE'])

for x, y, y_pred in zip(x_test, y_test, predictions):
  show_plot(x,y,y_pred)

 # Gated Recurrent Unit (GRU) Network

In [0]:
# GRU
inp = Input(shape=x_train.shape[-2:])
x = GRU(128, return_sequences=True)(inp)
x = GRU(64, return_sequences=True)(x)
x = Flatten()(x)
x = Dense(64)(x)
x = Dense(forecast_horizon)(x)
model = keras.Model(inputs=inp, outputs=x)

model.compile(optimizer='adam', loss='mae')
print(model.summary())

history = model.fit(x_train, y_train,
          batch_size=256,
          epochs=5,
          verbose=1,
          validation_data=(x_test, y_test))

plt.figure()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Training', 'Validation'])

predictions = model.predict(x_test)
mae = mean_absolute_error(y_test, predictions)
results['MAE']['GRU'] = mae
results['Y_PRED']['GRU'] = predictions
print(pd.DataFrame(results)['MAE'])

for x, y, y_pred in zip(x_test, y_test, predictions):
  show_plot(x,y,y_pred)

# Temporal Convolutional Network (TCN)

In [0]:
# TCN
inp = Input(shape=x_train.shape[-2:])
x = TCN(nb_filters=64, kernel_size=3, nb_stacks=1, dilations=[1,2,4,8,16,32,64])(inp)
x = Dense(forecast_horizon)(x)
model = keras.Model(inputs=inp, outputs=x)

model.compile(optimizer='adam', loss='mae')
print(model.summary())

history = model.fit(x_train, y_train,
          batch_size=256,
          epochs=5,
          verbose=1,
          validation_data=(x_test, y_test))

plt.figure()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Training', 'Validation'])

predictions = model.predict(x_test)
mae = mean_absolute_error(y_test, predictions)
results['MAE']['TCN'] = mae
results['Y_PRED']['TCN'] = predictions
print(pd.DataFrame(results)['MAE'])

for x, y, y_pred in zip(x_test, y_test, predictions):
  show_plot(x,y,y_pred)

# Convolutional Neural Network (CNN)

In [0]:
# CNN
inp = Input(shape=x_train.shape[-2:])
x = Conv1D(128, 7, activation='relu', padding='same')(inp)
x = MaxPool1D(pool_size=2)(x)
x = tf.keras.layers.Conv1D(64, 5, activation='relu', padding='same')(inp)
x = tf.keras.layers.MaxPool1D(pool_size=2)(x)
x = tf.keras.layers.Flatten()(x)
x = Dense(forecast_horizon)(x)
model = keras.Model(inputs=inp, outputs=x)

model.compile(optimizer='adam', loss='mae')
print(model.summary())

history = model.fit(x_train, y_train,
          batch_size=256,
          epochs=5,
          verbose=1,
          validation_data=(x_test, y_test))

plt.figure()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Training', 'Validation'])

predictions = model.predict(x_test)
mae = mean_absolute_error(y_test, predictions)
results['MAE']['CNN'] = mae
results['Y_PRED']['CNN'] = predictions
print(pd.DataFrame(results)['MAE'])

for x, y, y_pred in zip(x_test, y_test, predictions):
  show_plot(x,y,y_pred)

# Multi-Layer Perceptron (MLP)

In [0]:
# MLP
inp = Input(shape=x_train.shape[-2:])
x = Flatten()(inp)
x = Dense(128)(x)
x = Dense(64)(x)
x = Dense(32)(x)
x = Dense(forecast_horizon)(x)
model = keras.Model(inputs=inp, outputs=x)

model.compile(optimizer='adam', loss='mae')
print(model.summary())

history = model.fit(x_train, y_train,
          batch_size=256,
          epochs=5,
          verbose=1,
          validation_data=(x_test, y_test))

plt.figure()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Training', 'Validation'])

predictions = model.predict(x_test)
mae = mean_absolute_error(y_test, predictions)
results['MAE']['MLP'] = mae
results['Y_PRED']['MLP'] = predictions
print(pd.DataFrame(results)['MAE'])

for x, y, y_pred in zip(x_test, y_test, predictions):
  show_plot(x,y,y_pred)

# Analysis of results

In [0]:
pd.DataFrame(results)['MAE']

In [0]:
for i, (x, y)  in enumerate(zip(x_test, y_test)):
  y_pred=None
  fig, ax = plt.subplots(1,1, figsize=(15,5))
  ax.plot(x, 'o-', c=colors(3), markersize=3.5, label='History')
  ax.plot([m for m in range(x.shape[0],x.shape[0]+y.shape[0])], y, 'x-', c=colors(0), markersize=3.5, label='True Future')
  for j, (model_name, preds) in enumerate(results['Y_PRED'].items()): 
    y_pred = preds[i]
    ax.plot([m for m in range(x.shape[0],x.shape[0]+y.shape[0])], y_pred, 'o-', c=colors(2+j), alpha=0.7, markersize=3.5, label=model_name)
  ax.legend()